# Labolatorium nr 6 - Search engine

#### Patryk Klatka
#### 19 kwietnia 2023

## Wstęp

Celem tego labolatorium było stworzenie wyszukiwarki dokumentów oraz zbadanie wpływu SVD na wyniki wyszukiwań. Do konstrukcji zbioru dokumentów zostało wykorzystane oficjalne API Wikipedii. Zostały pobrane artykuły z kategorii Computed, gdzie ich liczebność wyniosła 7391. Następnie dla każdego artykułu został utworzony bag-of-words, uprzednio wykonując proces lemmatyzacji i usuwając tzw. stop words. Łącznie bag-of-words zawierało 46382 słowa. Następnie zostały wyznaczone macierze z zastosowaniem IDF oraz SVD. Dla SVD parametr k został wybrany na podstawie prób w trakcie wykonywania ćwiczenia.

Sprawozdanie zostało napisane w Jupyter Notebooku, w celu przedstawienia nie tylko wniosków z przeprowadzonego labolatorium, ale również kodu, który został wykorzystany do jego wykonania.

## Import bibliotek oraz ich konfiguracja

In [1]:
import dask.array as da
from spacy.lang.en.stop_words import STOP_WORDS
import spacy
import pickle
import numpy as np
import re
from multiprocessing.pool import ThreadPool
from collections import Counter
import os
import scipy as sp
import requests
nlp = spacy.load("en_core_web_sm")
global_folder_name = ""  # Used to store data in different folders for different runs

## Preprocessing

### Pobranie artykułów z danej kategorii

W celu znalezienia artykułów w kategorii została napisana prosta funkcja, która przechodzi rekurencyjnie po podkategoriach danej kategorii i zwraca listę artykułów w niej zawartych.

In [2]:
def get_articles_from_category(start_category, max_article_number=500):
    current_articles = 0
    articles = []
    stack = [start_category]
    visited = set([start_category])

    while current_articles < max_article_number and stack:
        category = stack.pop()
        # Get articles
        response = requests.get(
            f"https://en.wikipedia.org/w/api.php?action=query&list=categorymembers&cmtitle={category}&cmlimit=500&format=json")

        response = response.json()

        # Get query
        query = response["query"]['categorymembers']

        for article in query:
            if article["ns"] == 0:
                articles.append(article)
                current_articles += 1
                if current_articles == max_article_number:
                    stack = []
                    break
            elif article["ns"] == 14 and article["title"] not in visited:
                stack.append(article["title"])
                visited.add(article["title"])

    # Filter duplicates
    pageid_set = set()
    filtered_articles = []
    for article in articles:
        if article["pageid"] not in pageid_set:
            pageid_set.add(article["pageid"])
            filtered_articles.append(article)

    start_category = start_category.replace(":", "-")
    with open(f'./pages-from-categories/{start_category}.pickle', 'wb') as f:
        pickle.dump(filtered_articles, f)

    return filtered_articles

### Pobieranie tekstu z artykułu o podanym id

Po pobraniu listy artykułów, została napisana funkcja, która pobiera tekst artykułu o podanym id, parsuje go i tworzy bag-of-words odpowiednio przed tym stosując lemmatyzację i usuwanie tzw. stop words. Każdy artykuł został zapisany w osobnym pliku, gdzie nazwa pliku to id artykułu.

In [3]:
def get_pages(filename, number_of_articles=100, thread_pool_size=10):
    if number_of_articles < thread_pool_size:
        thread_pool_size = number_of_articles

    with open(filename, 'rb') as f:
        data = pickle.load(f)

    data = list(map(lambda x: x["pageid"], data))

    # Get random titles
    random_titles = np.random.choice(data, number_of_articles, replace=False)

    # Split titles into chunks
    random_titles = np.array_split(random_titles, thread_pool_size)

    errors = 0

    def create_dict(pageids):
        errors = 0
        for pageid in pageids:
            try:
                response = requests.get(
                    f"https://en.wikipedia.org/w/api.php?format=json&action=query&prop=extracts&explaintext&exsectionformat=wiki&redirects=1&pageids={pageid}")

                page = response.json()
                article_id = list(page['query']['pages'].keys())[0]
                content = re.sub(
                    r"={2} .+", "", page['query']['pages'][article_id]['extract'])

                # Tokenize text
                tokens = re.findall(
                    "[^\W\d_]+", content)

                # Lemmatize tokens and remove stop words
                tokens = [token.lemma_.lower() for token in nlp(
                    " ".join(tokens)) if token.lemma_ not in STOP_WORDS]

                # Count tokens
                counted_tokens = Counter(tokens)
                counted_tokens = {k: v for k,
                                v in counted_tokens.items() if v > 1}

                # Get summary
                summary_request = requests.get(
                    f"https://en.wikipedia.org/w/api.php?format=json&exintro&action=query&prop=extracts&explaintext&exsectionformat=wiki&redirects=1&pageids={article_id}&exsentences=2")

                summary = summary_request.json()

                result = {
                    "title": page['query']['pages'][article_id]['title'],
                    "pageid": article_id,
                    "link": f"https://en.wikipedia.org/wiki?curid={article_id}",
                    "tokens": counted_tokens,
                    "tokensNumber": sum(counted_tokens.values()),
                    "summary": summary["query"]["pages"][article_id]["extract"],
                }

                with open(f"./parsed-articles{global_folder_name}/article-{article_id}.pickle", "wb") as f:
                    pickle.dump(result, f)

            except Exception as e:
                print(f"ERROR: {pageid} - {e}")
                errors += 1
        return errors

    with ThreadPool(thread_pool_size) as pool:
        # Call a function on each item in a list and handle results
        for result in pool.map(create_dict, random_titles):
            # Count errors
            errors += result

    return errors

### Utworzenie macierzy rzadkiej

Następnie, po pobraniu artykułów, została utworzona funkcja, która tworzy macierz rzadką, gdzie wiersze odpowiadają słowom z bag-of-words, a kolumny artykułom, odpowiednio z możliwością zastosowania IDF lub SVD. W przypadku tej funkcji, parametr k określa ile najmniejszych wartości osobliwych ma zostać usuniętych.

In [4]:
def build_sparse_matrix(n=-1, use_idf=False, k=-1):
    # Get n articles
    i = 0
    articles = []
    words_in_articles = {}
    for file in os.listdir('./parsed-articles'):
        if os.path.getsize(f"./parsed-articles/{file}") == 0:
            continue

        with open(f"./parsed-articles/{file}", "rb") as f:
            article = pickle.load(f)
            for word in article["tokens"].keys():
                if word not in words_in_articles:
                    words_in_articles[word] = 0
                words_in_articles[word] += 1

            articles.append(
                {"title": article["title"], "pageid": article["pageid"], "link": article["link"], "summary": article["summary"]})
            i += 1

        if n != -1 and i == n:
            break

    word_set = list(words_in_articles.keys())

    # Map words to indexes
    word_set_index = {k: v for v, k in enumerate(word_set)}

    # Create sparse matrix
    matrix = sp.sparse.lil_matrix((len(articles), len(word_set)))

    # Fill sparse matrix
    for i, article in enumerate(articles):
        if os.path.getsize(f"./parsed-articles/article-{article['pageid']}.pickle") == 0:
            continue

        with open(f"./parsed-articles/article-{article['pageid']}.pickle", "rb") as f:
            article = pickle.load(f)
            for word in article["tokens"]:
                matrix[i, word_set_index[word]] = article["tokens"][word]

    idf_text = ""
    svd_text = ""

    # Normalize matrix using Inverse Document Frequency
    if use_idf:
        for i in range(matrix.shape[0]):
            matrix[i] = matrix[i] * \
                np.log(matrix.shape[0] / words_in_articles[word_set[i]])
        idf_text = "-idf"

    # Remove noises using SVD
    if k != -1:
        matrix = da.asarray(matrix.transpose())
        u, s, vt = da.linalg.svd_compressed(matrix, k=min(matrix.shape)-k)
        # u, s, vt = sp.sparse.linalg.svds(matrix, k=k)
        matrix = None  # Free memory
        matrix = sp.sparse.csr_matrix(u * s @ vt).transpose()
        svd_text = "-svd"

    # Transpose matrix
    matrix = matrix.transpose()

    # Get better memory representation
    matrix = matrix.tocsr()

    # Calculate matrix norm
    matrix_norm = sp.sparse.linalg.norm(matrix, axis=0)

    # Save objects to pickle files
    with open(f"./calculated-components/matrix{svd_text}{idf_text}.pickle", "wb") as f:
        pickle.dump((matrix, matrix_norm), f)

    with open("./calculated-components/word_set.pickle", "wb") as f:
        pickle.dump(word_set, f)

    with open("./calculated-components/articles.pickle", "wb") as f:
        pickle.dump(articles, f)

    print("Number of words:", len(word_set))
    print("Number of articles:", len(articles))

    return matrix, word_set, articles

## Utworzenie macierzy

Wygenerowano cztery macierze, dla każdej z nich wykorzystano inną metodę tworzenia macierzy. Wszystkie macierze zostały zapisane do plików, aby nie musieć ich ponownie tworzyć. Parametr k został wybrany na podstawie wcześniejszych prób, gdzie zostały wyznaczone wartości k dla różnych wartości SVD i wybrana taka, która dawała najlepsze wyniki.

In [5]:
# # Category:Computing
# category_name = "Category:Computing"
# x = get_articles_from_category(category_name, 10000)

# category_name = category_name.replace(':', '-')
# with open(f"./pages-from-categories/{category_name}.pickle", 'rb') as f:
#     data = pickle.load(f)

# # global_folder_name = f"-{category_name}"
# global_folder_name = f""
# os.mkdir(f"./parsed-articles{global_folder_name}")
# res = get_pages(
#     f'./pages-from-categories/{category_name}.pickle', len(data), 1000)

In [6]:
# articles = build_sparse_matrix()
# articles = build_sparse_matrix(use_idf=True)
# articles = build_sparse_matrix(k=3500, use_idf=True)
# articles = build_sparse_matrix(k=3500, use_idf=True)

## Zapytania

Każde zapytanie w postaci ciągów znaków jest zamieniane na wektor 0/1. Następnie dla każdego artykułu wyznaczana jest podobieństwo między wektorem zapytania a wektorem artykułu. Wyniki są sortowane malejąco i zwracane zgodnie z parametrami funkcji.

In [7]:
def get_results_from_query_vector(query, k=10, matrix_filename="matrix"):
    # Get matrix
    with open(f"./calculated-components/{matrix_filename}.pickle", "rb") as f:
        matrix, matrix_norm = pickle.load(f)

    # Get word set
    with open("./calculated-components/word_set.pickle", "rb") as f:
        word_set = pickle.load(f)

    # Get articles
    with open("./calculated-components/articles.pickle", "rb") as f:
        articles = pickle.load(f)

    # Create query vector
    query_vector = sp.sparse.lil_matrix((len(word_set), 1))

    # Fill query vector
    for word in re.findall("[^\W\d_]+", query.lower()):
        if word in word_set:
            query_vector[word_set.index(word)] = 1

    # Get vector norm
    query_vector_norm = sp.sparse.linalg.norm(query_vector)
    norm = query_vector_norm * matrix_norm

    # Calculate probabilities
    probabilities = (query_vector.T @ matrix) / norm

    probabilities = [(probabilities[0, i], i) for i in range(matrix.shape[1])]

    # Sort probabilities
    probabilities = list(map(lambda x: (x[0], articles[x[1]]), filter(lambda x: not np.isnan(x[0]) and np.isfinite(
        x[0]) and x[0] > 0, sorted(probabilities, key=lambda x: x[0], reverse=True))))

    # Get top k results
    return probabilities[: k]

### Przykładowe zapytanie nr 1

In [10]:
search_string = "algorithm"
print("Matrix results:")
print(get_results_from_query_vector(search_string, k=3, matrix_filename="matrix"))
print("Matrix-SVD results:")
print(get_results_from_query_vector(search_string, k=3, matrix_filename="matrix-svd"))
print("Matrix-IDF results:")
print(get_results_from_query_vector(search_string, k=3, matrix_filename="matrix-idf"))
print("Matrix-SVD-IDF results:")
print(get_results_from_query_vector(search_string, k=3, matrix_filename="matrix-svd-idf"))

Matrix results:
[(0.7730206825239258, {'title': 'Algorithmic curation', 'pageid': '73233518', 'link': 'https://en.wikipedia.org/wiki?curid=73233518', 'summary': 'Algorithmic curation is the curation (organizing and maintaining a collection) of online media using computer algorithms. Examples include search engine algorithms and social media algorithms.'}), (0.5784924073369803, {'title': 'Jewels of Stringology', 'pageid': '63902887', 'link': 'https://en.wikipedia.org/wiki?curid=63902887', 'summary': 'Jewels of Stringology: Text Algorithms is a book on algorithms for pattern matching in strings and related problems. It was written by Maxime Crochemore and Wojciech Rytter, and published by World Scientific in 2003.'}), (0.5, {'title': 'Algorithms Unlocked', 'pageid': '46573763', 'link': 'https://en.wikipedia.org/wiki?curid=46573763', 'summary': 'Algorithms Unlocked is a book by Thomas H. Cormen about the basic principles and  applications of computer algorithms. The book consists of ten c

### Przykładowe zapytanie nr 2

In [50]:
search_string = "compression"
print("Matrix results:")
print(get_results_from_query_vector(search_string, k=3, matrix_filename="matrix"))
print("Matrix-SVD results:")
print(get_results_from_query_vector(search_string, k=3, matrix_filename="matrix-svd"))
print("Matrix-IDF results:")
print(get_results_from_query_vector(search_string, k=3, matrix_filename="matrix-idf"))
print("Matrix-SVD-IDF results:")
print(get_results_from_query_vector(search_string, k=3, matrix_filename="matrix-svd-idf"))

Matrix results:


KeyboardInterrupt: 

### Przykładowe zapytanie nr 3

In [49]:
search_string = "queue"
print("Matrix results:")
print(get_results_from_query_vector(search_string, k=3, matrix_filename="matrix"))
print("Matrix-SVD results:")
print(get_results_from_query_vector(search_string, k=3, matrix_filename="matrix-svd"))
print("Matrix-IDF results:")
print(get_results_from_query_vector(search_string, k=3, matrix_filename="matrix-idf"))
print("Matrix-SVD-IDF results:")
print(get_results_from_query_vector(search_string, k=3, matrix_filename="matrix-svd-idf"))

Matrix results:
[(0.6324555320336759, {'title': 'Layered queueing network', 'pageid': '35685456', 'link': 'https://en.wikipedia.org/wiki?curid=35685456', 'summary': 'In queueing theory, a discipline within the mathematical theory of probability, a layered queueing network (or rendezvous network) is a queueing network model where the service time for each job at each service node is given by the response time of a queueing network (and those service times in turn may also be determined by further nested networks). Resources can be nested and queues form along the nodes of the nesting structure.'}), (0.6047078979069521, {'title': 'Command queue', 'pageid': '5187054', 'link': 'https://en.wikipedia.org/wiki?curid=5187054', 'summary': 'In computer science, a command queue is a queue for enabling the delay of command execution, either in order of priority, on a first-in first-out basis, or in any order that serves the current purpose. Instead of waiting for each command to be executed before

## Wnioski